# Association Rule Learning (ARL)
In today's world where the number of customers and transactions are increasing, it has become more valuable to create meaningful results from data and for developing marketing strategies. Revealing hidden patterns in the data in order to be able to compete better and maximize profit in the face of intense competition in the market, and to establish value-oriented long-term relationships with customers, makes a great contribution to determining marketing strategies.

However, the development of rule-based strategies is no longer possible in big data world, offering the right product to the right customer at the right time; it forms the basis of cross-selling and loyalty programs within the scope of customer retention and increasing lifetime value. Therefore, it has been crucial point for companies making product offers by using these patterns of association and developing effective marketing strategies Market Basket analysis is one of the association rule applications. It allows us to predict the products that customers tend to buy in the future by developing a pattern from their past behavior and habits.

There are different algorithms to be used for Association Rules Learning. One of them is the Apriori algorithm. In this project, product association analysis will be handled with “Apriori Algorithm” and the most suitable product offers will be made for the customer who is in the sales process, using the sales data of an e-commerce company.

How Apriori works ?
To construct association rules between elements or items, the algorithm considers 3 important factors which are, support, confidence and lift. Each of these factors is explained as follows:

Support:

The support of item I is defined as the ratio between the number of transactions containing the item I by the total number of transactions.

Confidence:

This is measured by the proportion of transactions with item I1, in which item I2 also appears. The confidence between two items I1 and I2, in a transaction is defined as the total number of transactions containing both items I1 and I2 divided by the total number of transactions containing I1.

Lift:

Lift is the ratio between the confidence and support.


# The mlxtend.frequent_patterns
Is a module provides a set of functions for generating frequent itemsets and association rules from transactional data using the Apriori algorithm. The main usage of this module is to identify the frequent itemsets in a transactional dataset, which are sets of items that appear together frequently in transactions. These frequent itemsets can then be used to generate association rules, which describe the relationships between different items in the dataset.

# IMPORTING NEEDED LIBRARIES

In [18]:
!pip install chart_studio
!pip install cufflinks
!pip install plotly
!pip install missingno

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.4/64.4 KB 2.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 18.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [19]:

import matplotlib.pyplot as plt
import seaborn   as sns
import pylab
import calendar
from   scipy import stats as sci
import missingno as msno
from   datetime import datetime

# Standard plotly imports
import chart_studio.plotly as py
import plotly.graph_objs   as go
from   plotly.offline import iplot, init_notebook_mode

# Using plotly + cufflinks in offline mode
import cufflinks
cufflinks.go_offline(connected=True)
init_notebook_mode(connected=True)

# Set global theme
cufflinks.set_config_file(world_readable=True, theme="pearl")

# Display all cell outputs
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

import plotly.io as pio
pio.renderers.default = "colab"

# Warnings
import warnings 
%matplotlib inline
warnings.filterwarnings('ignore')

In [1]:
import pandas as pd 
import numpy as np 
from mlxtend.frequent_patterns import association_rules, apriori
 
# Drive connect to get the data.
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data_preprocessed.csv')
df = data.copy()

In [3]:
df.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session,count_user,...,apparel.glove,appliances.environment.air_conditioner,appliances.environment.vacuum,appliances.personal.hair_cutter,furniture.bathroom.bath,furniture.living_room.cabinet,furniture.living_room.chair,other,sport.diving,stationery.cartrige
0,2019-10-01 00:00:00 UTC,cart,5773203,1487580005134238553,other,runail,2.62,463240011,26dd6e6e-4dac-4778-8d2c-92e149dab885,7,...,0,0,0,0,0,0,0,1,0,0
1,2019-10-01 00:00:03 UTC,cart,5773353,1487580005134238553,other,runail,2.62,463240011,26dd6e6e-4dac-4778-8d2c-92e149dab885,7,...,0,0,0,0,0,0,0,1,0,0
2,2019-10-01 00:00:07 UTC,cart,5881589,2151191071051219817,other,lovely,13.48,429681830,49e8d843-adf3-428b-a2c3-fe8bc6a307c9,34,...,0,0,0,0,0,0,0,1,0,0
3,2019-10-01 00:00:07 UTC,cart,5723490,1487580005134238553,other,runail,2.62,463240011,26dd6e6e-4dac-4778-8d2c-92e149dab885,7,...,0,0,0,0,0,0,0,1,0,0
4,2019-10-01 00:00:15 UTC,cart,5881449,1487580013522845895,other,lovely,0.56,429681830,49e8d843-adf3-428b-a2c3-fe8bc6a307c9,34,...,0,0,0,0,0,0,0,1,0,0


# Rules for products purchased on the same session

### Creating a function which returns 0 if the item was not in that transaction & 1 means item present in that transaction

In [4]:
def encode(x):
    if x<=0:
        return 0
    if x>=1:
        return 1

### Creating a dataframe contains the count of each product purchased in the same session

In [23]:
transactions_session = df[df['event_type'] == 'purchase'].groupby([ 'user_session','product_id',])['product_id'].count().reset_index(name ='Count')
transactions_session

,user_session,product_id,Count
0,000013d6-68a4-40cf-9452-6577dbfab515,5666407,1
1,000013d6-68a4-40cf-9452-6577dbfab515,5677403,1
2,000013d6-68a4-40cf-9452-6577dbfab515,5687138,1
3,000013d6-68a4-40cf-9452-6577dbfab515,5738763,1
4,000013d6-68a4-40cf-9452-6577dbfab515,5739125,1
...,...,...,...
564183,ffff8529-58c1-4ec8-933e-77f535718061,5752164,1
564184,ffff8529-58c1-4ec8-933e-77f535718061,5839651,1
564185,ffff8529-58c1-4ec8-933e-77f535718061,5859403,1
564186,ffff8529-58c1-4ec8-933e-77f535718061,5875389,1


In [24]:
# Extract a sample from transactions dataframe
transactions_session=transactions_session[:10000]
transactions_session

,user_session,product_id,Count
0,000013d6-68a4-40cf-9452-6577dbfab515,5666407,1
1,000013d6-68a4-40cf-9452-6577dbfab515,5677403,1
2,000013d6-68a4-40cf-9452-6577dbfab515,5687138,1
3,000013d6-68a4-40cf-9452-6577dbfab515,5738763,1
4,000013d6-68a4-40cf-9452-6577dbfab515,5739125,1
...,...,...,...
9995,0442fcd4-9995-478e-9589-a8053b29e1a5,5809911,1
9996,0442fcd4-9995-478e-9589-a8053b29e1a5,5809918,1
9997,0442fcd4-9995-478e-9589-a8053b29e1a5,5816178,1
9998,0442fcd4-9995-478e-9589-a8053b29e1a5,5820417,1


In [25]:
# making a mxn matrice where m=transaction and n=items and each row represents whether the item was in the transaction or not
my_basket = transactions_session.pivot_table(index='user_session', columns='product_id',values='Count',aggfunc='sum').fillna(0)
my_basket.head()

product_id,3762,3774,3945,3978,4102,4185,4203,4210,4229,4382,...,5906199,5907073,5907834,5907837,5907870,5907877,5907879,5908180,5908230,5908820
user_session,,,,,,,,,,,,,,,,,,,,,
000013d6-68a4-40cf-9452-6577dbfab515,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00008836-6d96-4580-8527-173d4e1b2889,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0001981a-ee7c-4542-bf81-2cea59d963ab,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0002ad82-84d0-40be-8d1a-b48de3ef8fe3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0003ece6-f25b-4d8c-92ef-107c6e351e44,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
# applying the function to the dataset
my_basket_sets = my_basket.applymap(encode)
my_basket_sets.head()

product_id,3762,3774,3945,3978,4102,4185,4203,4210,4229,4382,...,5906199,5907073,5907834,5907837,5907870,5907877,5907879,5908180,5908230,5908820
user_session,,,,,,,,,,,,,,,,,,,,,
000013d6-68a4-40cf-9452-6577dbfab515,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
00008836-6d96-4580-8527-173d4e1b2889,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0001981a-ee7c-4542-bf81-2cea59d963ab,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0002ad82-84d0-40be-8d1a-b48de3ef8fe3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0003ece6-f25b-4d8c-92ef-107c6e351e44,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Apply the algorithm on a sample

In [27]:
# frequent itemset
frequent_items = apriori(my_basket_sets, min_support = 0.0045,use_colnames = True)
frequent_items

,support,itemsets
0,0.007042,(4497)
1,0.006162,(4554)
2,0.005282,(4591)
3,0.005282,(4600)
4,0.007042,(4607)
...,...,...
182,0.005282,"(5854897, 5700046)"
183,0.007042,"(5849033, 5751422)"
184,0.005282,"(5816172, 5809910)"
185,0.005282,"(5833330, 5833325)"


In [28]:
# now making the rules from frequent itemset generated above
rules = association_rules(frequent_items, metric = "lift", min_threshold = 1)
rules.sort_values('confidence', ascending = False, inplace = True)
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
13,(5843546),(5843545),0.007042,0.008803,0.005282,0.750000,85.200000,0.005220,3.964789
12,(5843545),(5843546),0.008803,0.007042,0.005282,0.600000,85.200000,0.005220,2.482394
8,(5816172),(5809910),0.009683,0.035211,0.005282,0.545455,15.490909,0.004941,2.122535
11,(5833325),(5833330),0.010563,0.014085,0.005282,0.500000,35.500000,0.005133,1.971831
6,(5849033),(5751422),0.016725,0.021127,0.007042,0.421053,19.929825,0.006689,1.690781


In [30]:
rules.groupby(['antecedents','consequents'])['confidence'].mean().sort_values(ascending = False).head(10).iplot(
    kind="bar",
    yTitle="CONFIDENCE",
    linecolor="black",
    title="TOP 10 RULES",
)

How to read the above data ?
antecedents and consequents -> The IF component of an association rule is known as the antecedent. The THEN component is known as the consequent. The antecedent and the consequent are disjoint; they have no items in common.
 
antecedent support -> This measure gives an idea of how frequent antecedent is in all the transactions.

consequent support -> This measure gives an idea of how frequent consequent is in all the transactions.

support -> This measure gives an idea of how frequent ItemSet is in all the transactions. 

confidance -> This measure defines the likeliness of occurrence of consequent on the cart given that the cart already has the antecedents.

lift -> This measure defines the likeliness of occurrence of consequent on the cart given that the cart already has the antecedent, but controlling the popularity of consequent. 

This result shows that for example if the user purchased the product with id=5843546 he may also purchase with product with id=5843545 in the same session.

# Rules for products purchased by the same user

### Creating a dataframe contains the count of each product purchased by the same user


In [31]:
transactions_user = df[df['event_type'] == 'purchase'].groupby([ 'user_id','product_id',])['product_id'].count().reset_index(name ='Count')
transactions_user

,user_id,product_id,Count
0,9794320,5724621,1
1,9794320,5809910,1
2,9794320,5811668,1
3,9794320,5812033,1
4,10079204,5614842,1
...,...,...,...
549663,579925377,5759491,1
549664,579925377,5810673,1
549665,579925377,5813882,1
549666,579925377,5837126,1


In [32]:
# Extract a sample from transactions dataframe
transactions_user=transactions_user[:100000]
transactions_user

,user_id,product_id,Count
0,9794320,5724621,1
1,9794320,5809910,1
2,9794320,5811668,1
3,9794320,5812033,1
4,10079204,5614842,1
...,...,...,...
9995,233399486,5897517,1
9996,233399486,5898106,1
9997,233399486,5898116,1
9998,233399486,5898371,1


In [33]:
# making a mxn matrice where m=transaction and n=items and each row represents whether the item was in the transaction or not
my_basket = transactions_user.pivot_table(index='user_id', columns='product_id',values='Count',aggfunc='sum').fillna(0)
my_basket.head()

product_id,3936,3959,3978,4006,4092,4131,4185,4200,4210,4216,...,5908219,5908227,5908228,5908229,5908230,5908231,5908232,5908234,5908238,5908240
user_id,,,,,,,,,,,,,,,,,,,,,
9794320,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10079204,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10280338,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15781728,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19762782,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [34]:
# applying the function to the dataset
my_basket_sets = my_basket.applymap(encode)
my_basket_sets.head()

product_id,3936,3959,3978,4006,4092,4131,4185,4200,4210,4216,...,5908219,5908227,5908228,5908229,5908230,5908231,5908232,5908234,5908238,5908240
user_id,,,,,,,,,,,,,,,,,,,,,
9794320,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10079204,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10280338,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15781728,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19762782,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Apply the algorithm on a sample

In [35]:
# frequent itemset
frequent_items = apriori(my_basket_sets, min_support = 0.0045,use_colnames = True)
frequent_items

,support,itemsets
0,0.006702,(3978)
1,0.006702,(4185)
2,0.009383,(4497)
3,0.005362,(4600)
4,0.006702,(4768)
...,...,...
501,0.005362,"(5816166, 5809910, 5809911)"
502,0.005362,"(5809911, 5809910, 5854823)"
503,0.005362,"(5809912, 5816170, 5809910)"
504,0.005362,"(5809912, 5816178, 5809910)"


In [36]:
# now making the rules from frequent itemset generated above
rules = association_rules(frequent_items, metric = "lift", min_threshold = 1)
rules.sort_values('confidence', ascending = False, inplace = True)
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
156,"(5809910, 5816166)",(5809911),0.005362,0.021448,0.005362,1.0,46.625000,0.005247,inf
117,(5815571),(5815568),0.005362,0.005362,0.005362,1.0,186.500000,0.005333,inf
182,"(5833325, 5865798)",(5833326),0.005362,0.018767,0.005362,1.0,53.285714,0.005261,inf
116,(5815568),(5815571),0.005362,0.005362,0.005362,1.0,186.500000,0.005333,inf
181,"(5865798, 5833326)",(5833325),0.005362,0.029491,0.005362,1.0,33.909091,0.005204,inf


In [37]:
rules.groupby(['antecedents','consequents'])['confidence'].mean().sort_values(ascending = False).head(10).iplot(
    kind="bar",
    yTitle="CONFIDENCE",
    linecolor="black",
    title="TOP 10 RULES",
)

# Apply the algorithm on the whole dataset

In [38]:
x=np.empty(56)
start=0
end=10000
all_rules=pd.DataFrame()
for i in x:
    transactions = df[df['event_type'] == 'purchase'].groupby([ 'user_session','product_id',])['product_id'].count().reset_index(name ='Count')
    transactions=transactions[start:end]
    my_basket = transactions.pivot_table(index='user_session', columns='product_id',values='Count',aggfunc='sum').fillna(0)
    my_basket_sets = my_basket.applymap(encode)
    frequent_items = apriori(my_basket_sets, min_support = 0.0045,use_colnames = True)
    rules = association_rules(frequent_items, metric = "lift", min_threshold = 1)
    rules.sort_values('confidence', ascending = False, inplace = True)
    all_rules=all_rules.append(rules,ignore_index=True)
    start=start+10000
    end=end+10000
all_rules    

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(5843546),(5843545),0.007042,0.008803,0.005282,0.750000,85.200000,0.005220,3.964789
1,(5843545),(5843546),0.008803,0.007042,0.005282,0.600000,85.200000,0.005220,2.482394
2,(5816172),(5809910),0.009683,0.035211,0.005282,0.545455,15.490909,0.004941,2.122535
3,(5833325),(5833330),0.010563,0.014085,0.005282,0.500000,35.500000,0.005133,1.971831
4,(5849033),(5751422),0.016725,0.021127,0.007042,0.421053,19.929825,0.006689,1.690781
...,...,...,...,...,...,...,...,...,...
1055,(5751422),(5751383),0.020052,0.015693,0.006975,0.347826,22.164251,0.006660,1.509271
1056,(5833325),(5833330),0.018309,0.021796,0.006103,0.333333,15.293333,0.005704,1.467306
1057,(5809910),(5809912),0.032258,0.022668,0.009590,0.297297,13.115385,0.008859,1.390819
1058,(5833330),(5833325),0.021796,0.018309,0.006103,0.280000,15.293333,0.005704,1.363460
